In [15]:
!pip install torch_geometric

import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5, 3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

In [11]:
pip install torch

In [31]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

GCN Test Accuracy: 0.72


## Summary:

### Q&A
*   What is the final test accuracy after implementing the enhancements?
    The final test accuracy for the enhanced GCN model, incorporating hyperparameter tuning and dropout, was 78%.
*   If the 80% target is not met, provide further suggestions for optimization, referencing the previously discussed enhancement strategies.
    Since the 80% target was not met, a comprehensive list of optimization suggestions was provided, including more extensive hyperparameter searches, increasing model depth/width, advanced regularization, early stopping, graph data augmentation, node feature engineering, and exploring different GNN variants (GAT, GraphSAGE, APPNP), and addressing class imbalance.

### Data Analysis Key Findings
*   The `SimpleGCN` model was successfully modified to include a `torch.nn.Dropout` layer after the first `ReLU` activation for regularization.
*   The model was trained with the following hyperparameters: a learning rate of 0.01, 16 hidden dimensions, 200 training epochs, and a dropout rate of 0.5.
*   The final test accuracy achieved by the enhanced GCN model with these settings was 0.78 (78%), which is below the target accuracy of 80%.

### Insights or Next Steps
*   The current GCN model, even with initial hyperparameter tuning and dropout, requires further optimization to meet the 80% accuracy target on the Cora dataset.
*   A more systematic and extensive hyperparameter search (e.g., using Grid Search or Bayesian Optimization) and exploring alternative GNN architectures like Graph Attention Networks (GAT) or GraphSAGE are crucial next steps for performance improvement.
